In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
import numpy as np
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from paramiko import SSHClient
from scp import SCPClient
from subprocess import call

In [2]:
dat_2 = pd.read_pickle("two_players_data")
dat_3 = pd.read_pickle("three_players_data")
dat_4 = pd.read_pickle("four_players_data")

In [3]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw"
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

factions = {
    "terrans",
    "lantids",
    "hadsch-hallas",
    "ivits",
    "baltaks",
    "geodens",
    "xenos",
    "gleens",
    "ambas",
    "taklons",
    "bescods",
    "firaks",
    "itars",
    "nevlas"
}

final_scorings = {
    "planetType",
    "structure", 
    "sector", 
    "satellite", 
    "gaia",
    "structureFed"
}

In [4]:
def find_optimal_brute(scores, score_list, cur_sum, r_num):
    if r_num == (len(scores[0]) - 1):
        max_found = -100000000000
        max_i = 0
        for i in range(len(scores)):
            if i not in score_list:
                if max_found < scores[i][r_num]:
                    max_found = scores[i][r_num]
                    max_i = i
        score_list.append(max_i)
        return score_list, cur_sum + max_found
    else:
        best_sum = -1000000000
        best_list = []
        for i in range(len(scores)):
            if i not in score_list:
                s_c = score_list[:]
                s_c.append(i)
                l, s = find_optimal_brute(scores, s_c, cur_sum + scores[i][r_num], r_num + 1)
                if s > best_sum:
                    best_sum = s
                    best_list = l
        return best_list, best_sum

In [28]:
def make_round_scoring_heatmap(dat_x, num_players):
    #score_list = list(round_scorings.values())
    score_list = []
    for key in round_scorings.keys():
        score_list.append(round_scorings[key])
    
    # clean dat_x
    clean_pos = []
    for pos in range(num_players):
        dat = dat_x[dat_x['pos_' + str(pos + 1) + '_dropped'] == False]
        # dat = dat[dat['pos_' + str(pos + 1) + '_elo'] > 200]
        clean_pos.append(dat)
    dat_clean = pd.concat(clean_pos, ignore_index=True)
    
    for faction in factions:
        factions_pos = []
        for pos in range(num_players):
            factions_pos.append(dat_clean[dat_clean["pos_" + str(pos + 1) +"_faction"] == faction])
        factions_in = pd.concat(factions_pos, ignore_index=True)
        
        scores = [[None] * 6 for i in range(len(score_list))]
        scores_overall = [[None] * 6 for i in range(len(score_list))]
        scores_diff = [[None] * 6 for i in range(len(score_list))]
        ave_pos = [[None] * 6 for i in range(len(score_list))]
        tot_pos = [[None] * 6 for i in range(len(score_list))]
        ave_pos_min_faction = [[None] * 6 for i in range(len(score_list))]
        
        for pos in range(num_players):
            df = factions_pos[pos]
            dat = clean_pos[pos]
            for i, scoring in enumerate(score_list):
                for j in range(6):
                    s = df[df['round_' + str(j + 1) + '_scoring'] == scoring]
                    s_all = dat[dat['round_' + str(j + 1) + '_scoring'] == scoring]
                    if scores[i][j] is None:
                        scores[i][j] = s['pos_' + str(pos + 1) + '_score_round' + str(j + 1)].mean()
                        scores_overall[i][j] = s_all['pos_' + str(pos + 1) + '_score_round' + str(j + 1)].mean()
                        if pos == 0:
                            ave_pos[i][j] = len(s)
                        tot_pos[i][j] = len(s)
                    else:
                        scores[i][j] += s['pos_' + str(pos + 1) + '_score_round' + str(j + 1)].mean()
                        scores_overall[i][j] += s_all['pos_' + str(pos + 1) + '_score_round' + str(j + 1)].mean()
                        if pos == 0:
                            ave_pos[i][j] += len(s)
                        tot_pos[i][j] += len(s)
        
        faction_ave_pos = 0
        tot_pos_f = 0
        for pos in range(num_players):
            if pos == 0:
                faction_ave_pos += len(factions_pos[pos])
            tot_pos_f += len(factions_pos[pos])
        faction_ave_pos /= tot_pos_f
            
        # print(scores)
        # print(scores_overall)
        for i in range(len(score_list)):
            for j in range(6):
                if scores[i][j] is not None:
                    scores[i][j] /= num_players
                    scores_overall[i][j] /= num_players
                    scores_diff[i][j] = scores[i][j] - scores_overall[i][j]
                    ave_pos[i][j] /= tot_pos[i][j]
                    ave_pos_min_faction[i][j] = ave_pos[i][j] - faction_ave_pos
                    ave_pos_min_faction[i][j] *= 100
                    ave_pos[i][j] *= 100
        
        rounds = ['round 1', 'round 2', 'round 3', 'round 4', 'round 5', 'round 6']
        r, tot = find_optimal_brute(scores, [], 0, 0)
        
        text = [[''] * 6 for i in range(len(score_list))]
        for i, index in enumerate(r):
            text[index][i] = str(i + 1)
        
        score_map = go.Heatmap(
            z = scores,
            x = rounds,
            y = score_list,
            # zmin = 9,
            # zmid = 0,
            # zmax = 27,
            colorscale = 'Greens',
            colorbar_x=0.45,
            text=text, 
            texttemplate="%{text}"
        )
        
        
        pos_map = go.Heatmap(
            z = ave_pos_min_faction,
            x = rounds,
            y = score_list, 
            # zmin = 9,
            zmid = 0,
            # zmax = 27,
            colorscale = 'RdBu',
            text=ave_pos
        )
        
        title = "Round scoring preferences for " + faction + ". Best round scoring combination average: " + str(round(tot, 2))
        
    
        
        fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.15, subplot_titles=("average points made",  "\u0394 in victory probability (" + str(round(faction_ave_pos * 100, 2)) + "%)"), shared_yaxes=True)
        
        fig.append_trace(score_map, row=1, col=1)
        fig.append_trace(pos_map, row=1, col=2)
        # fig.update_yaxes(visible=False)
        fig.update_layout(title_text=title)
        fig.show()
                        

In [29]:
make_round_scoring_heatmap(dat_4, 4)